# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In the /data directory on the Jupyter server, there is a file called `allergies.json` that contains a list of patient allergies.  It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [41]:
import json
ALLERGIES_FILE="allergies.json"

In [42]:
# Put your solution here
#begin solution
import json
def allergy_count(json_file):
    """(list)->int
    This function takes a JSON file as input and returns the number of entries in the JSON file.

    >>> allergy_count(ALLERGIES_FILE), path to the JSON file containing allergy data.
        Returns the number of entries in the JSON file.
    """
    try:
        with open(json_file) as f:
            data = json.load(f)
            #if file not found at that location raise the FileNotFoundError
    except FileNotFoundError:
        print(f"Warning: file not found: {json_file}")
        return 0
    with open(json_file) as f:
        data = json.load(f)
    return len(data['entry'])
    #end solution

In [43]:
allergy_count(ALLERGIES_FILE)

4

In [44]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [45]:
# Put your solution here
#begin solution
import json
def patient_count(json_file):
    """(list)->int
      This function counts the number of unique patients in a JSON file and returns the number of unique patients.

      >>> patient_count(ALLERGIES_FILE), path to the JSON file containing allergy data.

      Returning the number of unique patients in the JSON file.

    """
    try:
        with open(json_file) as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Warning: file not found: {json_file}")
        return 0
    with open(json_file) as f:
        data = json.load(f)
        #counts the nnumber of patients (assuming each entery in the list is a patient)
    return len({entry['resource']['patient']['display'] for entry in data['entry']})# lenght of data here represents patient count
    #end solution

In [46]:
patient_count(ALLERGIES_FILE)

2

In [47]:
assert type(patient_count(ALLERGIES_FILE)) == int
assert patient_count(ALLERGIES_FILE) == 2

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [48]:
# Put your solution here
#begin solution
import json
def allergy_per_patient(json_file):
    """(list)->int
    This function takes a JSON file as input and returns a dictionary where the keys are patient names and the values are the corresponding allergy counts.

    >>> allergy_per_patient(ALLERGIES_FILE), path to the JSON file containing allergy data.

       Returniing a dictionary where the keys are patient names and the values are the corresponding allergy counts.
       {'Paul Boal': 1, 'Jason Argonaut': 3}

    """
    patient_allergies = {} #initializes a dictionary to store patient allergies
    try:
        with open(json_file) as f:
            data = json.load(f)
        for entry in data['entry']:
            try:
                patient = entry['resource']['patient']['display']
                if patient not in patient_allergies:
                  patient_allergies[patient] = 0
                patient_allergies[patient] += len(entry['resource'].get('reaction', []))
                #raises a keyerror if the dictionary key doesn't exist
            except KeyError:
                print(f"Warning: skipping entery due to missing key:{entry}")
            except TypeError:
                pass
    except:
        pass
    return patient_allergies
    #end solution


In [49]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

In [50]:
assert type(allergy_per_patient(ALLERGIES_FILE)) == dict
assert allergy_per_patient(ALLERGIES_FILE) == {'Paul Boal': 1, 'Jason Argonaut': 3}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [51]:
# Put your solution
#begin solution
ALLERGIES_FILE="allergies.json"
import json
def allergy_list(json_file):
    """(list)->int
  This function extracts allergy information from a JSON file and returns a list of lists.
  Each inner list contains patient, substance, and reaction information.

  >>> allergy_list(ALLERGIES_FILE), path to the JSON file containing allergy data.
      A list of lists, where each inner list contains patient, substance, and reaction information is returned.
   """
    result = []
    try:
        with open(json_file) as f:
            data = json.load(f)

        for entry in data['entry']:
            try:
                patient = entry['resource']['patient']['display']
                substance = entry['resource']['substance']['coding'][0]['display']
                reaction = entry['resource']['reaction'][0]['manifestation'][0]['text']
                result.append([patient, substance, reaction])

            except KeyError:
                print(f"Warning: skipping entery due to missing key:{entry}")
            except TypeError:
                 pass
    except FileNotFoundError:
        print(f"file not found: {json_file}")
    # sort the result list by patient name and substance to ensure consistency in order
    result.sort()
    return result
    #end solution

In [52]:
allergy_list(ALLERGIES_FILE)

[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]

In [53]:
assert allergy_list(ALLERGIES_FILE) == [['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising']]


### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  You can solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [54]:
# Put your solution here
#begin solution
import json
def allergy_reaction(json_file, patient, substance):
    """(list)->int
    This function extracts allergy information from a JSON file and returns the reaction for a specific patient and substance.

     >>> allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') and a path to the JSON file containing allergy data with
       Name of the patient and Name of the substance.

      Returning the reaction for the specified patient and substance, or None if not found.
    """
    # tries to open and load the json file
    try:
        with open(json_file) as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Warning: file not found: {json_file}")
        return None

    with open(json_file, 'r') as f:
        data = json.load(f)
    result = None

    for entry in data['entry']:  # Check if the patient and substance match the specified inputs.
        if entry['resource']['patient']['display'] == patient and entry['resource']['substance']['text'] == substance:
            result = entry['resource']['reaction'][0]['manifestation'][0]['text']
            break # exitts the loop if the reaction is found

    return result
    #end solution

In [55]:
allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G')

'Hives'

In [56]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to save this notebook file back to GitHub.  To do that in Google Colab:
1. File -> Save a Copy in GitHub
2. Make sure your HDS5210 repository is selected
3. Make sure the file name includes the week number like this: `week06/week06_assignment_2.ipynb`
4. Add a commit message that means something

**Be sure week names are lowercase and use a two digit week number!!**

**Be sure you use the same file name provided by the instructor!!**

